## "Querying Source"


In [0]:
%sql
select * from pyspark_cata.source.product;


In [0]:
df = spark.read.table("pyspark_cata.source.product")

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

In [0]:
df = spark.sql("select * from pyspark_cata.source.product")
df=df.withColumn("dedup",row_number().over(Window.partitionBy("id").orderBy(desc("UpdatedDate"))))

df = df.filter(df.dedup==1).drop("dedup")
display(df)

## "Upsert"

In [0]:
from delta.tables import DeltaTable
if len(dbutils.fs.ls("/Volumes/pyspark_cata/source/db_volume/product_sink/"))>0: 
    dlt_obj = DeltaTable.forPath(
        spark,
        "/Volumes/pyspark_cata/source/db_volume/product_sink/"
    )
    dlt_obj.alias("tgt").merge(
        df.alias("src"),
        "src.id = tgt.id"
    ).whenMatchedUpdateAll(
        condition="src.UpdatedDate >= tgt.UpdatedDate"
    ).whenNotMatchedInsertAll().execute()
    display(df)
else:
    df.write.format("delta")\
        .mode("overwrite")\
        .save("/Volumes/pyspark_cata/source/db_volume/product_sink/")